## Load libraries

In [ ]:
#For importing modules
import os, sys

#For file handling
from glob import glob
from os import listdir
from os.path import isfile, join
import json 

#For image processing
import PIL
import cv2
import nibabel as nib
import nibabel.processing 
from nibabel.processing import resample_to_output
import nilearn


#Image visualisation
import matplotlib.pyplot as plt

#Other
import numpy as np
import math
from random import sample

#Modeling
import tensorflow as tf
import scipy
import sklearn
from sklearn.model_selection import train_test_split
from nilearn.image import concat_imgs, mean_img, resample_img

import tensorflow as tflow
from tensorflow.keras.layers import Flatten
from keras.layers.core import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import precision_score, recall_score, confusion_matrix,classification_report

In [21]:
import tensorflow as tf
import scipy
import sklearn
from sklearn.model_selection import train_test_split
from nilearn.image import concat_imgs, mean_img, resample_img

import tensorflow as tflow
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input, BatchNormalization
from keras.layers.core import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import os
import glob

import nibabel as nib
import nilearn

import json 

import numpy as np
import math
from random import sample

from sklearn.utils.class_weight import compute_class_weight

import matplotlib as mpl
from ipywidgets import interact, widgets
from IPython.display import display
import matplotlib.pyplot as plt
#import warnings
from ipywidgets import IntSlider
from mpl_toolkits.axes_grid1 import ImageGrid

import lime
from lime import lime_image
from skimage.segmentation import mark_boundaries

In [22]:
sys.path.append('/path/to/2014_07_13_test')
import generate_input

## Load data and model

In [23]:
# load validation set
val_case = ['case_00233', 'case_00089', 'case_00050', 'case_00112', 'case_00258', 'case_00246', 'case_00157', 'case_00149','case_00184']
test_case = ['case_00221', 'case_00259', 'case_00087', 'case_00254', 'case_00098', 'case_00023', 'case_00041',
             'case_00080', 'case_00101', 'case_00164', 'case_00002', 'case_00110', 'case_00030', 'case_00068',
             'case_00026', 'case_00063', 'case_00006', 'case_00048', 'case_00250', 'case_00238', 
             'case_00111', 'case_00278', 'case_00133', 'case_00284', 'case_00282', 'case_00269', 'case_00039',
             'case_00033', 'case_00108', 'case_00175', 'case_00161', 'case_00256', 'case_00119', 'case_00286',
             'case_00077', 'case_00162', 'case_00270', 'case_00271', 'case_00285', 'case_00174', 'case_00147',
             'case_00215', 'case_00150', 'case_00052', 'case_00231', 'case_00198', 'case_00117', 'case_00138',
             'case_00211', 'case_00190', 'case_00248', 'case_00235', 'case_00049', 'case_00074', 'case_00107',
             'case_00218', 'case_00001', 'case_00193', 'case_00067', 'case_00072', 'case_00044',
             'case_00294', 'case_00298', 'case_00263', 'case_00038', 'case_00299', 'case_00249', 'case_00225',
             'case_00217', 'case_00178', 'case_00082', 'case_00035', 'case_00034', 'case_00047', 'case_00276',
             'case_00151', 'case_00226', 'case_00086', 'case_00176']

In [24]:
def generate_part_label_dict(labels_dict, list_of_cases):
    """ Generate lists of pathes to the right files """
    return {k: labels_dict[k] for k in list_of_cases}

def load_data(rootdir, img_dir, json_path, list_cases):
    """ Load data in the right format """
    val_path = generate_input.processed_image_paths(rootdir, img_dir)
    labels_dict = generate_input.malignant_labels_to_dict(json_path)
    labels_dict_short = generate_part_label_dict(labels_dict, list_cases)
    image_dict_short = generate_input.load_nifti_img_and_mask_as_numpy(val_path, list_cases)
    x_set, y_set = generate_input.fill_set(list_cases, labels_dict_short, image_dict_short, labels_dict)
    x_set = generate_input.adding_channel(x_set)
    return x_set, y_set

In [25]:
def adding_channel_dict(image_dict):
    """ Adds channels to each 3D image in the dictionary"""
    for key in image_dict:
        image_dict[key] = generate_input.adding_channel(image_dict[key])
    return image_dict


def empty_image_removal_dict(masks, images):
    """ Removes padding of 0 from images and masks """
    x_final_dict = {}
    
    common_keys = masks.keys() & images.keys()
    for key in common_keys:
        x_final = []

        for i, pair in enumerate(zip(masks[key], images[key])):
    
            if len(np.unique(pair[0])) > 1:
                x_final.append(pair[1])
            
        x_final = np.array(x_final)

        
        x_final_dict[key] = x_final
    return x_final_dict

def load_data_to_evaluate_by_majority_voting(rootdir, img_dir, json_path, list_cases):
    """ Loads data in the right format """
    val_path = generate_input.processed_image_paths(rootdir, img_dir)
    labels_dict = generate_input.malignant_labels_to_dict(json_path)
    labels_dict_short = generate_part_label_dict(labels_dict, list_cases)
    labels_dict_short = {key: float(value) for key, value in labels_dict_short.items()}
    
    image_dict_short = generate_input.load_nifti_img_and_mask_as_numpy(val_path, list_cases)
    image_dict_short = adding_channel_dict(image_dict_short)
    return image_dict_short, labels_dict_short

In [26]:
test_dict, test_label_dict = load_data_to_evaluate_by_majority_voting('./cropped-data/', 'images-64', '../kits21/kits21/data/kits.json', test_case)

In [27]:
test_dict_masks, test_label = load_data_to_evaluate_by_majority_voting('./cropped-data/', 'masks-64', '../kits21/kits21/data/kits.json', test_case)

In [28]:
test_dict_short = empty_image_removal_dict(test_dict_masks, test_dict)

In [29]:
# load json and create model
json_file = open('model64croped_segmented3.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = tf.keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model64croped_segmented3.h5")

In [30]:
# evaluate loaded model on test data
optimizer_fn = tf.keras.optimizers.experimental.RMSprop(learning_rate=0.001, jit_compile = False)
metrics_list = [tf.keras.metrics.AUC(name = 'auc'),
                tf.keras.metrics.BinaryAccuracy(name = 'accuracy')]
loaded_model.compile(optimizer = optimizer_fn, loss='binary_crossentropy', metrics= metrics_list)

## Application of the XAI techniques

In [32]:
# adapted from https://usmanr149.github.io/urmlblog/cnn/2020/05/01/Salincy-Maps.html
def get_saliency_map(image, model):
    image =tf.Variable(image, dtype = float)
    with tf.GradientTape() as tape:
        tape.watch(image)
        predictions = model(image) 
        loss = predictions[:, 0]
    
    gradient = tape.gradient(loss, image)
    gradient = tf.math.abs(gradient)
    gradient = np.max(gradient, axis=3)[0]
    
    # normalize between 0 and 1
    min_val, max_val = np.min(gradient), np.max(gradient)
    smap = (gradient - min_val) / (max_val - min_val + tf.keras.backend.epsilon())
    return smap

In [33]:
# taken from https://keras.io/examples/vision/grad_cam/
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):

    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])
    

    with tf.GradientTape() as tape:
        last_conv_layer_output, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        loss = predictions[:, pred_index]

    gradient = tape.gradient(loss, last_conv_layer_output)

    pooled_grads = tf.reduce_mean(gradient, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    
    return heatmap.numpy()

def display_gradcam(image, heatmap, alpha=0.4):
    
    image = np.squeeze(image)
    
    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = mpl.colormaps.get_cmap('jet')
    
    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((image.shape[1], image.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    
    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + image
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)
    
    return superimposed_img

In [34]:
def create_prediction_dict(image_dict, model):
    """ Create a dictionary with patient level predictions """
    pred_dict = {}
    for key in image_dict:
        y_pred = model.predict(image_dict[key])
        voting = np.round(np.sum(y_pred)/len(y_pred))
        if voting > 0.5:
            pred_dict[key] = 'Malignant'
        else:
            pred_dict[key] = 'Benign'
        
    return pred_dict

In [35]:
majority_prediction = create_prediction_dict(test_dict_short, loaded_model)

2023-05-14 23:45:52.035287: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 0s 67ms/step


In [36]:
def create_slice_prediction(slices, model):
    """ Creates slice prediction and converts it to textual explanation """
    slices = slices.reshape((1,) + slices.shape) 
    slices_pred = float(model.predict(slices, verbose=None))
    if slices_pred > 0.5:
        return 'Malignant', round(((slices_pred-0.5)/(1-0.5))*100, 2) 

    else:
        return 'Benign', round((((1-slices_pred) -0.5)/(1-0.5))*100, 2)  
    
def rename_label(label):
    if label == 1.0:
        return 'Malignant'
    else:
        return 'Benign'

In [37]:
# %%capture
# %matplotlib inline
model = loaded_model
last_conv_layer_name = "conv4_block6_3_conv"
dataset = test_dict_short
labels = test_label_dict
patient_widget = dataset.keys()
slices = IntSlider(min=0, max=50, value = 20)
prediction = majority_prediction
malignant_widget = widgets.RadioButtons(options=['Malignant class', 'Benign class'], description='Contribution to:', disabled=False)

def f(Patient, Slice_number, Malignant):
    
    try:
        # update how long the second widget
        slices.max =len(dataset[Patient]) 
    
        slice_prediction = create_slice_prediction(dataset[Patient][Slice_number], model)
        
        def predicted(img_array):
        # create a prediction function for the LIME model
            return model.predict(img_array, verbose=False)
        explainer = lime_image.LimeImageExplainer(random_state=42)
        explanation = explainer.explain_instance(dataset[Patient][Slice_number], predicted)
        
        print("Case # " + Patient + ' Label: ' + str(rename_label(labels[Patient]) ))
        print("Case was predicted as " + str(prediction[Patient]))
        print('This slice was predicited as ' + str(slice_prediction[0]) + ' with ' +  str(slice_prediction[1]) + ' % probability')
        
        
        # take image
        img_array = dataset[Patient][Slice_number:Slice_number+1]
        
        # create subplots
        fig = plt.figure(figsize = (11,6))
        grid = ImageGrid(fig, 111, nrows_ncols = (2,3), axes_pad = 0.4, cbar_location = "right",
                         cbar_mode="single", cbar_size="2%", cbar_pad=0.05)
        


        # create raw image
        ax1 = grid[0].imshow(dataset[Patient][Slice_number])
        
        #create guided backpropagation
        smap = get_saliency_map(img_array, model)
        ax2 = grid[1].imshow(smap, cmap="RdBu_r")
        
        # create grad-cam
        heatmap_cam = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
        grad_cam = display_gradcam(img_array, heatmap_cam, alpha=0.002)
        ax3 = grid[2].imshow(grad_cam, cmap="RdBu_r")
        plt.colorbar(ax3, cax=grid.cbar_axes[0])
        
        # create LIME
        
        heatmap_LIME = np.vectorize(dict(explanation.local_exp[0]).get)(explanation.segments)

        if Malignant == 'Malignant class':
            image, mask = explanation.get_image_and_mask(0, positive_only = True, negative_only = False, hide_rest=True,  num_features=1)
            ax5 = grid[4].imshow(heatmap_LIME, cmap="RdBu_r")
        else:
            image, mask = explanation.get_image_and_mask(0, positive_only = False, negative_only = True, hide_rest=True,  num_features=1)
            ax5 = grid[4].imshow(heatmap_LIME, cmap="RdBu")
        ax4 = grid[3].imshow(mark_boundaries(image, mask))  

    
        grid[0].title.set_text('Raw image')
        grid[1].title.set_text('Guided backpropagation')
        grid[2].title.set_text('Grad-CAM')
        #LIME part
        grid[3].title.set_text('Parts contrbuted the prediction')
        grid[4].title.set_text('Heatmap of all parts')
    
        grid[0].axis('off')
        grid[1].axis('off')
        grid[2].axis('off')
        grid[3].axis('off')
        grid[4].axis('off')
        grid[5].axis('off')
        
    except IndexError:
        # update how long the second widget
        slices.max =len(dataset[Patient])
        slices.value = len(dataset[Patient])
        print('Change slice number')

In [38]:
interact(f, Patient = patient_widget, Slice_number = slices, Malignant = malignant_widget)

interactive(children=(Dropdown(description='Patient', options=('case_00107', 'case_00077', 'case_00299', 'case…

<function __main__.f(Patient, Slice_number, Malignant)>